# 4
# Aries Basic Controller - Openmined PryVote Voter (Bob)

This tutorial runs through the issuer-api from the perspective of a holder. It should be run alongside the [openmind-pryvote-community issuer notebook](http://127.0.0.1:8888/notebooks/openmind-pryvote-community.ipynb) from Openmined PryVote Community (Alice)'s perspective.

If unfamiliar with the protocol it is worth reading through the [aries-rfs](https://github.com/hyperledger/aries-rfcs/tree/master/features/0036-issue-credential)


In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

# 5
## Register listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())
def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Attributes: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)

Handle Credentials
Credential exchange 19ee6c90-2e48-448f-aadf-e3fb709e05a1, role: holder, state: offer_received
Attributes: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]


# 11
## Check Credential Exchange Records

The agent will have at least one record if you have run through the [openmind-pryvote-community issuer notebook](http://127.0.0.1:8888/notebooks/openmind-pryvote-community.ipynb) notebook up until send credential.

In [4]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")


Credential exchange 19ee6c90-2e48-448f-aadf-e3fb709e05a1, role: holder, state: offer_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]


# OPTIONAL!
## Propose Credential - not to be executed as part of notebook

If unhappy with the attributes being offered (e.g. there is a mistake), the holder can send a proposal to the issuer for an alternative credential. The holder can also initiate the protocol with a proposal to the issuer, this is left out of scope in the tutorial, but should be possible to edit to achieve this if interested.

Note the proposal will create a new credential exchange record, the tutorial currently does not progress this to issued state but instead continues with the previous credential_exchange.

**Arguments (These can all be found in a credential exchange record)**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to False. **Not entirely sure about this one, believe its for logging. Also when set to True it throws an error**

In [ ]:
### NOT TO BE EXECUTED AS PART OF NOTEBOOK
connection_id = cred_record['connection_id']
schema_id = cred_record['schema_id']
cred_def_id = cred_record['credential_definition_id']
proposed_attributes = [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '0'}]

proposal_record = await agent_controller.issuer.send_proposal(connection_id, schema_id, cred_def_id, proposed_attributes)
proposal_record_id = proposal_record['credential_exchange_id']
proposal_state = proposal_record['state']
proposal_role = proposal_record['role']
proposal_attributes = proposal_record['credential_proposal_dict']['credential_proposal']['attributes']
print(f"Proposal credential exchange record {proposal_record_id}, role: {proposal_role}, state: {proposal_state}")
print(f"Attributes: {attributes}")


# 12
## Request Credential from Openmined PryVote Community (Alice) Issuer

If happy with the attributes being offered in the credential, then the holder requests the credential from the issuer to proceed with the issuance.

It is only possible to request a credential from an exchange when it is in the offer_received state

In [5]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 19ee6c90-2e48-448f-aadf-e3fb709e05a1, role: holder, state: request_sent
Handle Credentials
Credential exchange 19ee6c90-2e48-448f-aadf-e3fb709e05a1, role: holder, state: request_sent
Attributes: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]
Handle Credentials
Credential exchange 19ee6c90-2e48-448f-aadf-e3fb709e05a1, role: holder, state: credential_received
Attributes: [{'name': 'name', 'value': 'Bob'}, {'name': 'community_member', 'value': '1'}]


# 13
## Store the credential

Once the Openmined PryVote Community (Alice) issuer has responded to a request by sending the credential, the holder needs to store it to save the credential for later.

First check that the credential record is in the credential_received state

In [8]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

ClientResponseError: 404, message='Not Found', url=URL('http://bob-agent:8051/issue-credential/records/19ee6c90-2e48-448f-aadf-e3fb709e05a1')

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<run_in_terminal.<locals>.run() done, defined at /opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py:50> exception=UnsupportedOperation('fileno')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py", line 55, in run
    return func()
  File "/aries_basic_controller/utils.py", line 120, in <lambda>
    run_in_terminal(lambda: print_ext(*msg, color=color, **kwargs))
  File "/aries_basic_controller/utils.py", line 103, in print_ext
    print_formatted(FormattedText(msg), **kwargs)
  File "/aries_basic_controller/utils.py", line 83, in print_formatted
    prompt_toolkit.print_formatted_text(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/shortcuts/utils.py", line 112, in print_formatted_text
    output = get_app_session().output
  File "/opt/conda/lib/python3.7/site-packages/

In [9]:
response = await agent_controller.issuer.store_credential(cred_ex_id, "My OM Credential")
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

ClientResponseError: 500, message='Internal Server Error', url=URL('http://bob-agent:8051/issue-credential/records/19ee6c90-2e48-448f-aadf-e3fb709e05a1/store')

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<run_in_terminal.<locals>.run() done, defined at /opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py:50> exception=UnsupportedOperation('fileno')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py", line 55, in run
    return func()
  File "/aries_basic_controller/utils.py", line 120, in <lambda>
    run_in_terminal(lambda: print_ext(*msg, color=color, **kwargs))
  File "/aries_basic_controller/utils.py", line 103, in print_ext
    print_formatted(FormattedText(msg), **kwargs)
  File "/aries_basic_controller/utils.py", line 83, in print_formatted
    prompt_toolkit.print_formatted_text(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/shortcuts/utils.py", line 112, in print_formatted_text
    output = get_app_session().output
  File "/opt/conda/lib/python3.7/site-packages/

# 15
## Get Proof Request Records

In [10]:
response = await agent_controller.proofs.get_records()
print(response)

print('\n')

state = response['results'][0]["state"]
presentation_exchange_id = response['results'][0]['presentation_exchange_id']
presentation_request = response['results'][0]['presentation_request']

# print('Presentation Exchange ID\n')
# print(response['results'][0]['presentation_exchange_id'])
# pres_x_id = response['results'][0]['presentation_exchange_id']
# print('Presentation Request Object\n')
# print(response['results'][0]['presentation_request'])
# print('Requested Attributes\n')
# print(response['results'][0]['presentation_request']['requested_attributes'])
# requested_attribs = response['results'][0]['presentation_request']['requested_attributes']
# print('Requested Predicates\n')
# print(response['results'][0]['presentation_request']['requested_predicates'])
# requested_predicates = response['results'][0]['presentation_request']['requested_predicates']

{'results': [{'updated_at': '2020-08-20 11:46:48.002799Z', 'created_at': '2020-08-20 11:46:48.002799Z', 'initiator': 'external', 'thread_id': '3d55b9f7-8ad3-42f2-ac85-0d3cf27f7ea1', 'state': 'request_received', 'trace': False, 'connection_id': 'b7c869bc-3b04-48bd-b7cb-ad5e584a7ba2', 'presentation_request': {'name': 'Proof of Openmined Community Member', 'version': '1.0', 'requested_attributes': {'0_community_vote_uuid': {'name': 'community_vote'}}, 'requested_predicates': {'0_community_member_GE_uuid': {'name': 'community_member', 'p_type': '>=', 'p_value': 1, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'nonce': '17718931380412758176634'}, 'role': 'prover', 'presentation_exchange_id': '343fac22-7dc2-4f38-b495-8a83d36715ab'}]}




# 16
## Fetch Credentials for Proof Presentation

In [11]:
# presentation_exchange_id = '0b4e53ab-f1ba-4b7d-bd0f-bdab3a027626'
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")

# include self-attested attributes (not included in credentials)
credentials_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}

# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
print(credentials)

if credentials:
    for row in sorted(
        credentials,
        key=lambda c: dict(c["cred_info"]["attrs"]),
        reverse=True,
    ):
        for referent in row["presentation_referents"]:
            if referent not in credentials_by_reft:
                credentials_by_reft[referent] = row

for referent in presentation_request["requested_attributes"]:
    if referent in credentials_by_reft:
        revealed[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }
    else:
        self_attested[referent] = "True"

for referent in presentation_request["requested_predicates"]:
    if referent in credentials_by_reft:
        predicates[referent] = {
            "cred_id": credentials_by_reft[referent]["cred_info"][
                "referent"
            ]
        }

print("\nGenerate the proof")
request = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(request)
print("\n")
print(predicates)
print(revealed)
print(self_attested)

Received Request -> Query for credentials in the wallet that satisfy the proof request
[{'cred_info': {'referent': 'My OM Credential', 'attrs': {'community_member': '1', 'name': 'Bob'}, 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_community-member1:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'rev_reg_id': None, 'cred_rev_id': None}, 'interval': None, 'presentation_referents': ['0_community_member_GE_uuid']}]

Generate the proof
{'requested_predicates': {'0_community_member_GE_uuid': {'cred_id': 'My OM Credential'}}, 'requested_attributes': {}, 'self_attested_attributes': {'0_community_vote_uuid': 'True'}}


{'0_community_member_GE_uuid': {'cred_id': 'My OM Credential'}}
{}
{'0_community_vote_uuid': 'True'}


# 17
## Send a Proof Presentation

In [12]:
print("\nSend the proof to Openmined Community Agent (Alice)")
send_presentation = await agent_controller.proofs.send_presentation(presentation_exchange_id, revealed, self_attested, predicates)
print(response)


# await self.admin_POST(
#     (
#         "/present-proof/records/"
#         f"{presentation_exchange_id}/send-presentation"
#     ),
#     request,
# )



Send the proof to Openmined Community Agent (Alice)
{'results': [{'updated_at': '2020-08-20 11:46:48.002799Z', 'created_at': '2020-08-20 11:46:48.002799Z', 'initiator': 'external', 'thread_id': '3d55b9f7-8ad3-42f2-ac85-0d3cf27f7ea1', 'state': 'request_received', 'trace': False, 'connection_id': 'b7c869bc-3b04-48bd-b7cb-ad5e584a7ba2', 'presentation_request': {'name': 'Proof of Openmined Community Member', 'version': '1.0', 'requested_attributes': {'0_community_vote_uuid': {'name': 'community_vote'}}, 'requested_predicates': {'0_community_member_GE_uuid': {'name': 'community_member', 'p_type': '>=', 'p_value': 1, 'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}]}}, 'nonce': '17718931380412758176634'}, 'role': 'prover', 'presentation_exchange_id': '343fac22-7dc2-4f38-b495-8a83d36715ab'}]}


# 20
## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [13]:
response = await agent_controller.terminate()
print(response)

None
